# Разбор резюме

Рекрутерам и рекрутерским агенствам присылают резюме в формате pdf и docx. Чтобы вести базу кандидатов, из текстов нужно извлекать имя, возраст, контакты соискателя, предыдущий опыт, желаемую зарплату.

В этом примере рассматриваются только стандартные резюме, сгенерированные сайтом HH. Вариативность таких текстов очень низкая, всё оформлено строго по шаблону. С помощью Yargy-парсера можно написать программу, которая работает с произвольными резюме, но эта задача выходит за рамки данного примера.

![](thumb.png)

# Data

In [1]:
from glob import glob

texts = []
for path in glob('texts/*.txt'):
    with open(path) as file:
        text = file.read()
        texts.append(text)
        
intros = [_[:700] for _ in sorted(texts)]

In [2]:
from random import seed, sample

seed(41)
for text in sample(intros, 3):
    print(text)
    print('---' * 10)


Женщина, 44 года, родилась 31 марта 1972

Проживает: Смоленск
Гражданство: Россия, есть разрешение на работу: Россия
Не готова к переезду, не готова к командировкам

Желаемая должность и зарплата
Супервайзер отдела продаж
Продажи

• Розничная торговля
• Торговые сети

Занятость: полная занятость
График работы: полный день
Желательное время в пути до работы: не имеет значения

45 000
руб.

Опыт работы — 5 лет 2 месяца
Июнь 2011 — Июль
2016
5 лет 2 месяца

МинералТрансКомпани, ООО
Смоленск, mikei.ru

Директор по продажам
Контроль планов продаж. Снижение потерь. Организация работы торговой точки. Обучение
персонала. Открытие четырех новых магазинов с нуля.  Куст курирования 7-12 магазинов.

Об
------------------------------

Женщина, 38 лет, родилась 11 ноября 1977

Проживает: Санкт-Петербург, м. Купчино
Гражданство: Россия, есть разрешение на работу: Россия
Не готова к переезду, не готова к командировкам

Желаемая должность и зарплата
Главный  бухгалтер
Бухгалтерия, управленческий учет,

# Grammar

In [3]:
import json

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    rule, or_, and_, not_
)
from yargy.predicates import (
    eq, in_,
    type, normalized,
    dictionary,
    gte, lte
)
from yargy.pipelines import (
    pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy.tokenizer import MorphTokenizer, EOL


Intro = fact(
    'Intro',
    ['gender', 'age', 'birth', 'location',
     attribute('citizenship').repeatable(),
     attribute('permission').repeatable(),
     'relocation', 'travel',
     'position',
     attribute('subspecializations').repeatable(),
     'employment', 'schedule', 'commute',
     'salary'
    ]
)


INT = type('INT')
COMMA = eq(',')
COLON = eq(':')


def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def show_matches(rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        spans = [_.span for _ in matches]
        show_markup(line, spans)

## Gender

In [4]:
GENDERS = {
    'Женщина': 'female',
    'Мужчина': 'male'
}

GENDER = in_(GENDERS).interpretation(
    Intro.gender.custom(GENDERS.get)
)


show_matches(
    GENDER,
    'мужчина, Мужчина, мужчину',
    'Женщина'
)

мужчина, Мужчина, мужчину
         ───────         
Женщина
───────


## Age

In [5]:
AGE = rule(
    INT,
    normalized('год')
)


show_matches(
    AGE,
    '21 год, 25 лет'
)

21 год, 25 лет
──────  ──────


## Birth

In [6]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


AGE = rule(
    INT.interpretation(
        Intro.age.custom(int)
    ),
    normalized('год')
)

MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12
}

MONTH_NAME = dictionary(
    MONTHS
).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

DAY = and_(
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

YEAR = and_(
    gte(1900),
    lte(2100)
).interpretation(
    Date.year.custom(int)
)

DATE = rule(
    DAY,
    MONTH_NAME,
    YEAR
).interpretation(
    Date
)

BIRTH = rule(
    normalized('родиться'),
    DATE.interpretation(
        Intro.birth
    )
)


show_matches(
    BIRTH,
    'родился 21 февраля 1990',
    'родиться 32 сентябрь 2000',
    'родилась 01 июля 1917',
)

родился 21 февраля 1990
───────────────────────
родиться 32 сентябрь 2000
родилась 01 июля 1917
─────────────────────


## Socdem

In [7]:
SOCDEM = rule(
    GENDER, COMMA,
    AGE, COMMA,
    BIRTH
)


parser = Parser(SOCDEM)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])

Мужчина, 24 года, родился 30 ноября 1991
Женщина, 19 лет, родилась 30 мая 1997
Женщина, 58 лет, родилась 5 января 1958
Мужчина, 21 год, родился 18 октября 1994
Мужчина, 25 лет, родился 15 ноября 1990
Женщина, 18 лет, родилась 26 ноября 1997
Женщина, 27 лет, родилась 5 сентября 1988
Мужчина, 20 лет, родился 24 июля 1996
Мужчина, 38 лет, родился 24 мая 1978
Мужчина, 21 год, родился 8 июня 1995


## Location

In [8]:
# https://api.hh.ru/metro
# https://api.hh.ru/areas

def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')
            
            
METRO_STATIONS = set(load_lines('dicts/metro.txt'))
AREAS = set(load_lines('dicts/areas.txt'))
seed(10)
sample(sorted(METRO_STATIONS), 10), sample(sorted(AREAS), 10)

(['Проспект Мира',
  'Алма-Атинская',
  'Нахимовский проспект',
  'Парк Победы',
  'Проспект свободы',
  'Академгородок',
  'Достоевская',
  'Окружная',
  'Партизанская',
  'Козья слобода'],
 ['Старонижестеблиевская',
  'Голицыно',
  'Атаманская',
  'Перевальск',
  'Обоянь',
  'Кораблино',
  'Биробиджан',
  'Ижевск',
  'Чаплыгин',
  'Куйбышево'])

In [9]:
Location = fact(
    'Location',
    ['area', 'metro']
)


METRO = rule(
    'м', '.',
    pipeline(METRO_STATIONS).interpretation(
        Location.metro
    )
)

AREA = pipeline(AREAS).interpretation(
    Location.area
)

LOCATION = rule(
    AREA,
    rule(
        COMMA,
        METRO
    ).optional()
).interpretation(
    Location
)


show_matches(
    LOCATION,
    'место проживания: Москва, м. Парк Победы',
    'Киев, м.Киевская',
    'Россия',
    'в Москве',
    'м. парк победы',
    'на м. Кропоткинской',
)

место проживания: Москва, м. Парк Победы
                  ──────────────────────
Киев, м.Киевская
────────────────
Россия
──────
в Москве
м. парк победы
на м. Кропоткинской


In [10]:
TITLE = rule(
    normalized('проживает'), COLON
)

LIVES_AT = rule(
    TITLE,
    LOCATION
)


parser = Parser(LIVES_AT)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])

Проживает: Москва, м. Новогиреево
Проживает: Тула
Проживает: Липецк
Проживает: Зеленоград
Проживает: Новосибирск, м. Золотая нива
Проживает: Тамбов
Проживает: Москва
Проживает: Москва, м. Сухаревская
Проживает: Энгельс
Проживает: Казань, м. Проспект Победы


## Citizenship

In [11]:
TITLE = rule(
    'Гражданство', COLON
)

ITEM = AREA.interpretation(
    Intro.citizenship
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional()
)

CITIZENSHIP = rule(
    TITLE,
    LOCATIONS
)


show_matches(
    CITIZENSHIP,
    'Гражданство: Россия, Франция',
    'Гражданство: Россия, Франция, Украина',
)

Гражданство: Россия, Франция
────────────────────────────
Гражданство: Россия, Франция, Украина
────────────────────────────         


## Permission

In [12]:
TITLE = pipeline([
    'есть разрешение на работу:'
])

ITEM = AREA.interpretation(
    Intro.permission
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional().repeatable()
)

PERMISSION = rule(
    TITLE,
    LOCATIONS
)


show_matches(
    PERMISSION,
    'есть разрешение на работу: Россия, Франция, Украина',
    'есть разрешение на работу: Россия',
)

есть разрешение на работу: Россия, Франция, Украина
───────────────────────────────────────────────────
есть разрешение на работу: Россия
─────────────────────────────────


## Relocation

In [13]:
Relocation = fact(
    'Relocation',
    ['ready', attribute('where').repeatable()]
)

TYPES = {
    'готов к переезду': True,
    'хочу переехать': True,
    'не готов к переезду': False
}

IS_READY = morph_pipeline(TYPES).interpretation(
    Relocation.ready.normalized().custom(TYPES.get)
)

ITEM = AREA.interpretation(
    Relocation.where
)

LOCATIONS = rule(
    ITEM,
    rule(
        COMMA,
        ITEM
    ).optional().repeatable()
)

RELOCATION = rule(
    IS_READY,
    rule(
        COLON,
        LOCATIONS
    ).optional()
).interpretation(
    Relocation
).interpretation(
    Intro.relocation
)


show_matches(
    RELOCATION,
    'готов к переезду',
    'не готова к переезду',
    'готова к переезду: Россия, Украина, СНГ',
    'хочу переехать: Франция',
)

готов к переезду
────────────────
не готова к переезду
────────────────────
готова к переезду: Россия, Украина, СНГ
──────────────────────────────────     
хочу переехать: Франция
───────────────────────


## Travel

In [14]:
TYPES = {
    'готов к командировкам': True,
    'готов к редким командировкам': True,
    'не готов к командировкам': False
}

TRAVEL = morph_pipeline(TYPES).interpretation(
    Intro.travel.normalized().custom(TYPES.get)
)


show_matches(
    TRAVEL,
    'готова к командировкам',
    'не готов к редким командировкам',
)

готова к командировкам
──────────────────────
не готов к редким командировкам
   ────────────────────────────


## Position

In [15]:
# https://api.hh.ru/specializations

SPECIALIZATIONS = set(load_lines('dicts/specialization.txt'))
SUBSPECIALIZATIONS = set(load_lines('dicts/subspecialization.txt'))

seed(10)
sample(sorted(SPECIALIZATIONS), 10), sample(sorted(SUBSPECIALIZATIONS), 10)

(['Продажи',
  'Административный персонал',
  'Консультирование',
  'Медицина, фармацевтика',
  'Юристы',
  'Автомобильный бизнес',
  'Государственная служба, некоммерческие организации',
  'Маркетинг, реклама, PR',
  'Транспорт, логистика',
  'Домашний персонал'],
 ['Риски: рыночные',
  'Тренерский состав',
  'Делопроизводство',
  'Администратор баз данных',
  'Планирование, Размещение рекламы',
  'Охранник',
  'Кредиты: розничные',
  'Банкеты',
  'Информационные технологии, Интернет, Мультимедиа',
  'Электроэнергетика'])

In [16]:
TITLE = pipeline([
    'Желаемая должность и зарплата'
])

DOT = eq('•')

SUBTITLE = not_(DOT).repeatable().interpretation(
    Intro.position
)

SPECIALIZATION = pipeline(SPECIALIZATIONS)

SUBSPECIALIZATION = pipeline(SUBSPECIALIZATIONS)

ITEM = rule(
    DOT,
    or_(
        SPECIALIZATION,
        SUBSPECIALIZATION
    ).interpretation(
        Intro.subspecializations
    )
)

POSITION = rule(
    TITLE,
    SUBTITLE,
    ITEM.repeatable()
)


TOKENIZER = MorphTokenizer().remove_types(EOL)


parser = Parser(POSITION, tokenizer=TOKENIZER)
seed(10)
for text in sample(intros, 10):
    for match in parser.findall(text):
        start, stop = match.span
        print(text[start:stop])
        print('---')

Желаемая должность и зарплата
Водитель
Транспорт, логистика

• Автоперевозки
• Водитель
---
Желаемая должность и зарплата
Начинающий специалист
Начало карьеры, студенты

• Административный персонал
• Продажи
• Финансы, Банки, Инвестиции
---
Желаемая должность и зарплата
Администратор
Продажи

• Розничная торговля
• Торговые сети
---
Желаемая должность и зарплата
Продавец-консультант
Продажи

• Продавец в магазине
---
Желаемая должность и зарплата
Организатор мероприятий
Искусство, развлечения, масс-медиа

• Прочее
---
Желаемая должность и зарплата
Продавец-консультант
Продажи

• Розничная торговля
• Торговые сети
• Продавец в магазине
---
Желаемая должность и зарплата
Домработница
Домашний персонал

• домработница/домработник, Горничная
---
Желаемая должность и зарплата
Бармен, бармен кассир.
Начало карьеры, студенты

• Туризм, Гостиницы, Рестораны
---
Желаемая должность и зарплата
Менеджер по управлению товарными запасами
Транспорт, логистика

• Складское хозяйство
• Логистика
• Закуп

## Employment

In [17]:
TITLE = rule(
    'Занятость', COLON
)

TYPES = {
    'полная': 'full',
    'полная занятость': 'full',
    'частичная': 'part',
    'частичная занятость': 'part',
    'волонтерство': 'volunteer',
    'стажировка': 'intern',
    'проектная работа': 'project'
    
}

TYPE = pipeline(TYPES).interpretation(
    Intro.employment.normalized().custom(TYPES.get)
)

TYPES = rule(
    TYPE,
    rule(
        COMMA,
        TYPE
    ).optional().repeatable()
)

EMPLOYMENT = rule(
    TITLE,
    TYPES
)


show_matches(
    EMPLOYMENT,
    'Занятость: полная, частичная',
    'Занятость: стажировка',
)

Занятость: полная, частичная
────────────────────────────
Занятость: стажировка
─────────────────────


## Schedule

In [18]:
TITLE = pipeline([
    'График работы:'
])

TYPES = {
    'полный день': 'full',
    'сменный график': 'part',
    'вахтовый метод': 'vahta',
    'гибкий график': 'flex',
    'удаленная работа': 'remote',
    'стажировка': 'intern'
}

TYPE = morph_pipeline(TYPES).interpretation(
    Intro.schedule.normalized().custom(TYPES.get)
)

TYPES = rule(
    TYPE,
    rule(
        COMMA,
        TYPE
    ).optional().repeatable()
)

SCHEDULE = rule(
    TITLE,
    TYPES
)


show_matches(
    SCHEDULE,
    'График работы: полный день, удаленная работа',
    'График работы: стажировка',
)

График работы: полный день, удаленная работа
────────────────────────────────────────────
График работы: стажировка
─────────────────────────


## Commute

In [19]:
TITLE = pipeline([
    'Желательное время в пути до работы:',
])

TYPES = {
    'не более часа': '<1h',
    'не имеет значения': 'any',
    'не более полутора часов': '<1h30m'
}

TYPE = pipeline(TYPES).interpretation(
    Intro.commute.normalized().custom(TYPES.get)
)

COMMUTE = rule(
    TITLE,
    TYPE
)


show_matches(
    COMMUTE,
    'Желательное время в пути до работы: не более часа',
    'Желательное время в пути до работы: не имеет значения',
)

Желательное время в пути до работы: не более часа
─────────────────────────────────────────────────
Желательное время в пути до работы: не имеет значения
─────────────────────────────────────────────────────


## Money

In [20]:
Money = fact(
    'Money',
    ['amount', 'currency']
)


CURRENCIES = {
    'руб.': 'RUB',
    'грн.': 'GRN',
    'бел. руб.': 'BEL',
    'RUB': 'RUB',
    'EUR': 'EUR',
    'KZT': 'KZT',
    'USD': 'USD',
    'KGS': 'KGS'
}

CURRENCY = pipeline(CURRENCIES).interpretation(
    Money.currency.normalized().custom(CURRENCIES.get)
)


def normalize_amount(value):
    return int(value.replace(' ', ''))


AMOUNT = or_(
    rule(INT),
    rule(INT, INT),
).interpretation(
    Money.amount.custom(normalize_amount)
)

MONEY = rule(
    AMOUNT,
    CURRENCY
).interpretation(
    Money
).interpretation(
    Intro.salary
)


show_matches(
    MONEY,
    '1 500 руб.',
    '1 000 000 грн.',
    '5000 бел.руб.',
    '20 000 KGS',
)

1 500 руб.
──────────
1 000 000 грн.
  ────────────
5000 бел.руб.
─────────────
20 000 KGS
──────────


## Intro

In [21]:
INTRO = rule(
    SOCDEM,
    LIVES_AT,
    CITIZENSHIP, COMMA, PERMISSION,
    RELOCATION, COMMA, TRAVEL,
    POSITION,
    EMPLOYMENT,
    SCHEDULE,
    COMMUTE,
    MONEY
).interpretation(
    Intro
)


parser = Parser(INTRO, tokenizer=TOKENIZER)
seed(10)
for text in sample(intros, 10):
    matches = list(parser.findall(text))
    if matches:
        match = matches[0]
        fact = match.fact
        show_markup(text, fact.spans)
        show_json(fact.as_json)

Мужчина, 24 года, родился 30 ноября 1991
───────  ──               ── ────── ────
Проживает: Москва, м. Новогиреево
Гражданство: Россия, есть разрешение на работу: Россия
             ──────                             ──────
Не готов к переезду, не готов к командировкам
───────────────────  ────────────────────────
Желаемая должность и зарплата
Водитель
────────
Транспорт, логистика
────────────────────
• Автоперевозки
  ─────────────
• Водитель
  ────────
Занятость: полная занятость
           ────────────────
График работы: полный день
               ───────────
Желательное время в пути до работы: не имеет значения
                                    ─────────────────
45 000
──────
руб.
────
Опыт работы — 2 года 5 месяцев
Май 2015 —
Апрель 2016
1 год
ООО Панавто, г. Москва.
Диспетчер
Панавто- официальный представитель Mercedes-Benz в Москве.3 месяца 
работал
водителем-перегонщиком, основная деятельность перемещение автомобилей.
 Далее стал
диспетчером, к основным обязанностям переме